## Monte Carlo Process

In this cell, the configurations are set for using Spark in python 

In [63]:
import pyspark
sc = pyspark.SparkContext("local[*]", "MonteCarloIntegration")

In this cell, the required modules are imported

In [64]:
from random import random
from math import sin, cos
from operator import add
import scipy.integrate as integrate

The function in this cell produces a random point in R^2 and checks if the point is under the f(x)

In [65]:
def IsUnderFun(xMin, xMax, yMin, yMax):
    xRandom = xMin + (xMax-xMin) * random() 
    yRandom = yMin + (yMax - yMin) * random()
    return 1 if yRandom < Fun(xRandom) else 0 

In this cell, the integrated function, f(x), is defined

In [66]:
def Fun(x):
    return (1 + sin(x))/cos(x)

In this cell, the problem parameters are set up and the Spark program is executed 

In [68]:
if __name__ == "__main__":

    xMin = 0    # Minimum value for x
    xMax = 1    # Maximum value for x
    yMin = 0    # Minimum value for y
    yMax = 4    # Maximum value for y
    NumOfPoints = 1000  # number of points suggested in the Spark Assignment pdf
    
    result = integrate.quad(Fun, 0, 1)  # compute the integral using quadrature formulea
    print "Int[(1 + sin(x))/cos(x) dx, (%d, %d)]  = %2.8f is computed using scipy" % (xMin, xMax, result[0])
    print "Results using MC in Spark:"
    
    # compute the integral using Monte-Carlo (MC) with differet number of points starting from 1,000 up to 10,000,000
    while NumOfPoints < 1e8:  
        # do map and then reduce in order to obtain the number of points fallen under f(x)
        count = sc.parallelize(xrange(1, NumOfPoints + 1)).map(lambda _: IsUnderFun(xMin, xMax, yMin, yMax)).reduce(add)
        Integral = (xMax-xMin) * yMax * float(count)/float(NumOfPoints)   # compute the integral using MC integration 
        print "Number of points: %10d, Result = %2.8f" % (NumOfPoints, Integral) 
        NumOfPoints *= 10

Int[(1 + sin(x))/cos(x) dx, (0, 1)]  = 1.84181764 is computed using scipy
Results using MC in Spark:
Number of points:       1000, Result = 1.87200000
Number of points:      10000, Result = 1.85600000
Number of points:     100000, Result = 1.85648000
Number of points:    1000000, Result = 1.84466400
Number of points:   10000000, Result = 1.84188000


In [62]:
sc.stop()

### Additional comments: 
In order to have a better underestanding of the results, I have also computed the value of the integral using quadrature formulea and other number of points for MC process